<a href="https://colab.research.google.com/github/zizilnam/Kaggle_Ventilator_Pressure_Prediction/blob/main/VPP_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=b271feaf3b13b9681dc7b22b6ef1bb013738b5d18bc593027a606a80aba3f171
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import StackingRegressor
from bayes_opt import BayesianOptimization
from sklearn.linear_model import BayesianRidge

import lightgbm as lgb

from sklearn.metrics import mean_absolute_error
from math import sqrt

import joblib
import pickle

from tqdm import tqdm_notebook as tqdm
import time
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd /content/drive/MyDrive/Ventilator Pressure Prediction

/content/drive/MyDrive/Ventilator Pressure Prediction


In [ ]:
train = pd.read_csv('first_processed_train_df.csv')
test = pd.read_csv('first_processed_test_df.csv')

In [ ]:
train.drop(columns=['u_in_diff', 'u_out_diff'], inplace=True, axis=1)
test.drop(columns=['u_in_diff', 'u_out_diff'], inplace=True, axis=1)

In [ ]:
y_train = train['pressure']
train_ids = train['id']
train.drop(columns=['id','breath_id', 'pressure'], inplace=True, axis=1)
X_train = train

In [ ]:
ids = test['id']
test.drop(columns=['id','breath_id'], inplace=True, axis=1)

In [ ]:
columns = train.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
train_mae =mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
print("Train MAE: {:2.5f}".format(train_mae))
print("Test MAE: {:2.5f}".format(test_mae))

Train MAE: 3.07075
Test MAE: 3.07221


In [ ]:
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
params = {'alpha': alpha}

model=SGDRegressor()
grid_search = GridSearchCV(model, params, scoring = 'neg_mean_absolute_error', cv = 5)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
model = SGDRegressor(alpha=0.0001)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_mae =mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print("Train MAE: {:2.5f}".format(train_mae))
print("Test MAE: {:2.5f}".format(test_mae))

Train MAE: 11328940461895.34961
Test MAE: 11319976717520.04883


In [ ]:
params = {
            'n_estimators': [10, 50, 100, 150, 250, 500],
            'max_depth': [2, 4, 6, 8, 10],
            #'col_sample_rate': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
            'min_samples_split': [2, 4, 6, 8, 10],
            
}

model=RandomForestRegressor(n_jobs=-1)
random_search = GridSearchCV(model, params, scoring = 'neg_mean_absolute_error', cv = 5)
random_search.fit(X_train, y_train)
random_search.best_params_

In [ ]:
model = RandomForestRegressor(n_jobs=-1, n_estimators = 500, min_samples_split = 8, max_depth = 10)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_rmse = sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = sqrt(mean_squared_error(y_test, y_pred_test))

print("Train RMSE: {:2.5f}".format(train_rmse))
print("Test RMSE: {:2.5f}".format(test_rmse))